In [1]:
import pandas as pd
import string
import re
import nltk
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('wordnet')
stopword = nltk.corpus.stopwords.words("english")
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ckkoc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ckkoc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data = pd.read_csv("job_list_all.csv", sep = ",")
data.head()

,job_title,company_name,location,summary,job search
0,Data Analyst - YouTube,Webhelp Americas,"San Bruno, CAâ€¢Temporarily Remote",Strong grasp on data structure and data integr...,data analyst
1,Marketing Data Analyst (REMOTE),Molina Healthcare,"Long Beach, CA 90802â€¢Remote",Designs and implements processes and solutions...,data analyst
2,Entry Level Data Analyst,Insight Global,"Carlsbad, CA 92008",They will be responsible for data administrati...,data analyst
3,Entry Level - Data Engineer / Data Analyst (STEM),PCS Global Tech,"Poway, CAâ€¢Temporarily Remote","Provides plan with data, reporting, and analys...",data analyst
4,Data Analyst/Data Engineer,MT Global US INC,"Foster City, CA 94404","\* Owner of the core company data pipeline, re...",data analyst


In [3]:
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct
data["summary_clean"] = data["summary"].apply(lambda x: remove_punct(x))
data.head()

,job_title,company_name,location,summary,job search,summary_clean
0,Data Analyst - YouTube,Webhelp Americas,"San Bruno, CAâ€¢Temporarily Remote",Strong grasp on data structure and data integr...,data analyst,Strong grasp on data structure and data integr...
1,Marketing Data Analyst (REMOTE),Molina Healthcare,"Long Beach, CA 90802â€¢Remote",Designs and implements processes and solutions...,data analyst,Designs and implements processes and solutions...
2,Entry Level Data Analyst,Insight Global,"Carlsbad, CA 92008",They will be responsible for data administrati...,data analyst,They will be responsible for data administrati...
3,Entry Level - Data Engineer / Data Analyst (STEM),PCS Global Tech,"Poway, CAâ€¢Temporarily Remote","Provides plan with data, reporting, and analys...",data analyst,Provides plan with data reporting and analyses...
4,Data Analyst/Data Engineer,MT Global US INC,"Foster City, CA 94404","\* Owner of the core company data pipeline, re...",data analyst,Owner of the core company data pipeline respo...


In [4]:
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens
data["tokens"] = data["summary_clean"].apply(lambda x: tokenize(x.lower()))
data.head()

,job_title,company_name,location,summary,job search,summary_clean,tokens
0,Data Analyst - YouTube,Webhelp Americas,"San Bruno, CAâ€¢Temporarily Remote",Strong grasp on data structure and data integr...,data analyst,Strong grasp on data structure and data integr...,"[strong, grasp, on, data, structure, and, data..."
1,Marketing Data Analyst (REMOTE),Molina Healthcare,"Long Beach, CA 90802â€¢Remote",Designs and implements processes and solutions...,data analyst,Designs and implements processes and solutions...,"[designs, and, implements, processes, and, sol..."
2,Entry Level Data Analyst,Insight Global,"Carlsbad, CA 92008",They will be responsible for data administrati...,data analyst,They will be responsible for data administrati...,"[they, will, be, responsible, for, data, admin..."
3,Entry Level - Data Engineer / Data Analyst (STEM),PCS Global Tech,"Poway, CAâ€¢Temporarily Remote","Provides plan with data, reporting, and analys...",data analyst,Provides plan with data reporting and analyses...,"[provides, plan, with, data, reporting, and, a..."
4,Data Analyst/Data Engineer,MT Global US INC,"Foster City, CA 94404","\* Owner of the core company data pipeline, re...",data analyst,Owner of the core company data pipeline respo...,"[, owner, of, the, core, company, data, pipeli..."


In [5]:
def remove_stopwords(tokens):
    text = [word for word in tokens if word not in stopword]
    return text
data["no_stop"] = data["tokens"].apply(lambda x: remove_stopwords(x))
data.head()

,job_title,company_name,location,summary,job search,summary_clean,tokens,no_stop
0,Data Analyst - YouTube,Webhelp Americas,"San Bruno, CAâ€¢Temporarily Remote",Strong grasp on data structure and data integr...,data analyst,Strong grasp on data structure and data integr...,"[strong, grasp, on, data, structure, and, data...","[strong, grasp, data, structure, data, integri..."
1,Marketing Data Analyst (REMOTE),Molina Healthcare,"Long Beach, CA 90802â€¢Remote",Designs and implements processes and solutions...,data analyst,Designs and implements processes and solutions...,"[designs, and, implements, processes, and, sol...","[designs, implements, processes, solutions, as..."
2,Entry Level Data Analyst,Insight Global,"Carlsbad, CA 92008",They will be responsible for data administrati...,data analyst,They will be responsible for data administrati...,"[they, will, be, responsible, for, data, admin...","[responsible, data, administration, tasks, pul..."
3,Entry Level - Data Engineer / Data Analyst (STEM),PCS Global Tech,"Poway, CAâ€¢Temporarily Remote","Provides plan with data, reporting, and analys...",data analyst,Provides plan with data reporting and analyses...,"[provides, plan, with, data, reporting, and, a...","[provides, plan, data, reporting, analyses, en..."
4,Data Analyst/Data Engineer,MT Global US INC,"Foster City, CA 94404","\* Owner of the core company data pipeline, re...",data analyst,Owner of the core company data pipeline respo...,"[, owner, of, the, core, company, data, pipeli...","[, owner, core, company, data, pipeline, respo..."


In [6]:
def stemming(tnp):
    text = [ps.stem(word) for word in tnp]
    return text
data["stems"] = data["no_stop"].apply(lambda x: stemming(x))
data.head()

,job_title,company_name,location,summary,job search,summary_clean,tokens,no_stop,stems
0,Data Analyst - YouTube,Webhelp Americas,"San Bruno, CAâ€¢Temporarily Remote",Strong grasp on data structure and data integr...,data analyst,Strong grasp on data structure and data integr...,"[strong, grasp, on, data, structure, and, data...","[strong, grasp, data, structure, data, integri...","[strong, grasp, data, structur, data, integr, ..."
1,Marketing Data Analyst (REMOTE),Molina Healthcare,"Long Beach, CA 90802â€¢Remote",Designs and implements processes and solutions...,data analyst,Designs and implements processes and solutions...,"[designs, and, implements, processes, and, sol...","[designs, implements, processes, solutions, as...","[design, implement, process, solut, associ, wi..."
2,Entry Level Data Analyst,Insight Global,"Carlsbad, CA 92008",They will be responsible for data administrati...,data analyst,They will be responsible for data administrati...,"[they, will, be, responsible, for, data, admin...","[responsible, data, administration, tasks, pul...","[respons, data, administr, task, pull, data, i..."
3,Entry Level - Data Engineer / Data Analyst (STEM),PCS Global Tech,"Poway, CAâ€¢Temporarily Remote","Provides plan with data, reporting, and analys...",data analyst,Provides plan with data reporting and analyses...,"[provides, plan, with, data, reporting, and, a...","[provides, plan, data, reporting, analyses, en...","[provid, plan, data, report, analys, enabl, da..."
4,Data Analyst/Data Engineer,MT Global US INC,"Foster City, CA 94404","\* Owner of the core company data pipeline, re...",data analyst,Owner of the core company data pipeline respo...,"[, owner, of, the, core, company, data, pipeli...","[, owner, core, company, data, pipeline, respo...","[, owner, core, compani, data, pipelin, respon..."


In [7]:
def lemmatizing(tnp):
    text = [wn.lemmatize(word) for word in tnp]
    return text
data["lemmatize"] = data["no_stop"].apply(lambda x: lemmatizing(x))
data.to_csv("preprocessed_data.csv", sep=',')
data.head()

,job_title,company_name,location,summary,job search,summary_clean,tokens,no_stop,stems,lemmatize
0,Data Analyst - YouTube,Webhelp Americas,"San Bruno, CAâ€¢Temporarily Remote",Strong grasp on data structure and data integr...,data analyst,Strong grasp on data structure and data integr...,"[strong, grasp, on, data, structure, and, data...","[strong, grasp, data, structure, data, integri...","[strong, grasp, data, structur, data, integr, ...","[strong, grasp, data, structure, data, integri..."
1,Marketing Data Analyst (REMOTE),Molina Healthcare,"Long Beach, CA 90802â€¢Remote",Designs and implements processes and solutions...,data analyst,Designs and implements processes and solutions...,"[designs, and, implements, processes, and, sol...","[designs, implements, processes, solutions, as...","[design, implement, process, solut, associ, wi...","[design, implement, process, solution, associa..."
2,Entry Level Data Analyst,Insight Global,"Carlsbad, CA 92008",They will be responsible for data administrati...,data analyst,They will be responsible for data administrati...,"[they, will, be, responsible, for, data, admin...","[responsible, data, administration, tasks, pul...","[respons, data, administr, task, pull, data, i...","[responsible, data, administration, task, pull..."
3,Entry Level - Data Engineer / Data Analyst (STEM),PCS Global Tech,"Poway, CAâ€¢Temporarily Remote","Provides plan with data, reporting, and analys...",data analyst,Provides plan with data reporting and analyses...,"[provides, plan, with, data, reporting, and, a...","[provides, plan, data, reporting, analyses, en...","[provid, plan, data, report, analys, enabl, da...","[provides, plan, data, reporting, analysis, en..."
4,Data Analyst/Data Engineer,MT Global US INC,"Foster City, CA 94404","\* Owner of the core company data pipeline, re...",data analyst,Owner of the core company data pipeline respo...,"[, owner, of, the, core, company, data, pipeli...","[, owner, core, company, data, pipeline, respo...","[, owner, core, compani, data, pipelin, respon...","[, owner, core, company, data, pipeline, respo..."


In [8]:
pd.set_option('display.max_colwidth', 100) 
data = pd.read_csv("preprocessed_data.csv", sep=',')

In [9]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopword]
    return text

In [10]:
count_vect = CountVectorizer(analyzer=clean_text)
X_counts = count_vect.fit_transform(data['summary'])
print(X_counts.shape)
print(count_vect.get_feature_names())

(2559, 2960)
['', '1', '10', '1015', '10tb', '10â', '11000', '1159', '12', '12week', '13', '14', '15', '150170k', '1553', '180', '1st', '1stparti', '2', '20', '21st', '23', '24', '247', '25', '2year', '2â', '3', '30', '311', '34', '35', '36', '360degre', '362', '3d', '3year', '3ã', '4', '40', '401k', '41621', '430', '46', '48', '5', '510', '510â', '515', '57', '5g', '6', '6ã', '7', '70', '710', '730', '8', '810', 'ab', 'abil', 'abl', 'abreast', 'abstract', 'abstractâ', 'abus', 'academ', 'academia', 'academicwork', 'acceler', 'accentur', 'accept', 'access', 'accomplish', 'accord', 'account', 'accredit', 'accrual', 'accur', 'accuraci', 'achiev', 'achieveâ', 'acl', 'acquir', 'acquisit', 'across', 'acrossâ', 'act', 'action', 'activ', 'activevideo', 'activis', 'actuari', 'acumen', 'acycl', 'ad', 'adam', 'adapt', 'addit', 'address', 'adept', 'adequ', 'adher', 'adhoc', 'adjust', 'administr', 'adob', 'adobeomnitur', 'adobeâ', 'adopt', 'advanc', 'advancedâ', 'advantag', 'advertis', 'advertising

In [11]:
X_counts_df = pd.DataFrame(X_counts.toarray(), columns=count_vect.get_feature_names())
X_counts_df.head()

,,1,10,1015,10tb,10â,11000,1159,12,12week,...,zone,â,å,æ,ç,œbig,œcleanâ,œdata,œoper,œstoryâ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,1,2,1,0,0,0,0,0


In [12]:
ngram_vect = CountVectorizer(ngram_range=(2,2),analyzer=clean_text)
X_counts = ngram_vect.fit_transform(data['summary'])
print(X_counts.shape)
print(ngram_vect.get_feature_names())

(2559, 2960)
['', '1', '10', '1015', '10tb', '10â', '11000', '1159', '12', '12week', '13', '14', '15', '150170k', '1553', '180', '1st', '1stparti', '2', '20', '21st', '23', '24', '247', '25', '2year', '2â', '3', '30', '311', '34', '35', '36', '360degre', '362', '3d', '3year', '3ã', '4', '40', '401k', '41621', '430', '46', '48', '5', '510', '510â', '515', '57', '5g', '6', '6ã', '7', '70', '710', '730', '8', '810', 'ab', 'abil', 'abl', 'abreast', 'abstract', 'abstractâ', 'abus', 'academ', 'academia', 'academicwork', 'acceler', 'accentur', 'accept', 'access', 'accomplish', 'accord', 'account', 'accredit', 'accrual', 'accur', 'accuraci', 'achiev', 'achieveâ', 'acl', 'acquir', 'acquisit', 'across', 'acrossâ', 'act', 'action', 'activ', 'activevideo', 'activis', 'actuari', 'acumen', 'acycl', 'ad', 'adam', 'adapt', 'addit', 'address', 'adept', 'adequ', 'adher', 'adhoc', 'adjust', 'administr', 'adob', 'adobeomnitur', 'adobeâ', 'adopt', 'advanc', 'advancedâ', 'advantag', 'advertis', 'advertising

In [13]:
X_counts_df = pd.DataFrame(X_counts.toarray(), columns=ngram_vect.get_feature_names())
X_counts_df.head()

,,1,10,1015,10tb,10â,11000,1159,12,12week,...,zone,â,å,æ,ç,œbig,œcleanâ,œdata,œoper,œstoryâ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,1,2,1,0,0,0,0,0


In [14]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['summary'])
print(X_tfidf.shape)
print(tfidf_vect.get_feature_names())

(2559, 2960)
['', '1', '10', '1015', '10tb', '10â', '11000', '1159', '12', '12week', '13', '14', '15', '150170k', '1553', '180', '1st', '1stparti', '2', '20', '21st', '23', '24', '247', '25', '2year', '2â', '3', '30', '311', '34', '35', '36', '360degre', '362', '3d', '3year', '3ã', '4', '40', '401k', '41621', '430', '46', '48', '5', '510', '510â', '515', '57', '5g', '6', '6ã', '7', '70', '710', '730', '8', '810', 'ab', 'abil', 'abl', 'abreast', 'abstract', 'abstractâ', 'abus', 'academ', 'academia', 'academicwork', 'acceler', 'accentur', 'accept', 'access', 'accomplish', 'accord', 'account', 'accredit', 'accrual', 'accur', 'accuraci', 'achiev', 'achieveâ', 'acl', 'acquir', 'acquisit', 'across', 'acrossâ', 'act', 'action', 'activ', 'activevideo', 'activis', 'actuari', 'acumen', 'acycl', 'ad', 'adam', 'adapt', 'addit', 'address', 'adept', 'adequ', 'adher', 'adhoc', 'adjust', 'administr', 'adob', 'adobeomnitur', 'adobeâ', 'adopt', 'advanc', 'advancedâ', 'advantag', 'advertis', 'advertising

In [15]:
X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vect.get_feature_names())
X_tfidf_df.head()

,,1,10,1015,10tb,10â,11000,1159,12,12week,...,zone,â,å,æ,ç,œbig,œcleanâ,œdata,œoper,œstoryâ
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.084265,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.260476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.090396,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.057043,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.286558,0.573116,0.286558,0.0,0.0,0.0,0.0,0.0


In [16]:
print(X_tfidf_df.loc[(X_tfidf_df!=0).any(axis=1)])

                       1   10  1015  10tb  10â  11000  1159   12  12week  ...  \
0     0.000000  0.000000  0.0   0.0   0.0  0.0    0.0   0.0  0.0     0.0  ...   
1     0.084265  0.000000  0.0   0.0   0.0  0.0    0.0   0.0  0.0     0.0  ...   
2     0.000000  0.260476  0.0   0.0   0.0  0.0    0.0   0.0  0.0     0.0  ...   
3     0.090396  0.000000  0.0   0.0   0.0  0.0    0.0   0.0  0.0     0.0  ...   
4     0.057043  0.000000  0.0   0.0   0.0  0.0    0.0   0.0  0.0     0.0  ...   
...        ...       ...  ...   ...   ...  ...    ...   ...  ...     ...  ...   
2554  0.000000  0.000000  0.0   0.0   0.0  0.0    0.0   0.0  0.0     0.0  ...   
2555  0.078563  0.000000  0.0   0.0   0.0  0.0    0.0   0.0  0.0     0.0  ...   
2556  0.000000  0.000000  0.0   0.0   0.0  0.0    0.0   0.0  0.0     0.0  ...   
2557  0.080581  0.000000  0.0   0.0   0.0  0.0    0.0   0.0  0.0     0.0  ...   
2558  0.066091  0.000000  0.0   0.0   0.0  0.0    0.0   0.0  0.0     0.0  ...   

      zone    â         å  

In [17]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopword]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['summary'])
X_tfidf_feat = pd.DataFrame(X_tfidf.toarray())

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['summary'])
X_count_feat = pd.DataFrame(X_count.toarray())

In [18]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)# n_jobs=-1 for parallelizing search
gs_fit = gs.fit(X_count_feat, data['job search'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
8,24.660714,1.971125,0.302194,0.106425,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.828125,0.859375,0.876953,0.835938,0.857143,0.851507,0.017495,1
4,13.617596,0.385686,0.262098,0.032650,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.820312,0.849609,0.886719,0.839844,0.855186,0.850334,0.021722,2
10,13.769763,0.371098,0.230183,0.019920,None,150,"{'max_depth': None, 'n_estimators': 150}",0.832031,0.861328,0.878906,0.828125,0.843444,0.848767,0.018977,3
5,27.339192,0.689218,0.378787,0.015416,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.828125,0.857422,0.871094,0.832031,0.847358,0.847206,0.015933,4
7,13.622179,0.407909,0.225596,0.009821,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.826172,0.863281,0.880859,0.830078,0.833659,0.846810,0.021492,5


In [19]:
gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)# n_jobs=-1 for parallelizing search
gs_fit = gs.fit(X_tfidf_feat, data['job search'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
8,24.787888,1.472452,0.333707,0.086503,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.828125,0.845703,0.873047,0.835938,0.853229,0.847208,0.015475,1
10,13.408329,0.335598,0.211633,0.024154,None,150,"{'max_depth': None, 'n_estimators': 150}",0.822266,0.867188,0.865234,0.832031,0.845401,0.846424,0.017758,2
4,13.263716,0.247352,0.215823,0.020002,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.824219,0.851562,0.867188,0.826172,0.847358,0.843300,0.016204,3
7,13.427279,0.292919,0.225596,0.012653,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.818359,0.847656,0.880859,0.822266,0.847358,0.843300,0.022408,3
11,16.300391,2.115907,0.125663,0.032774,None,300,"{'max_depth': None, 'n_estimators': 300}",0.818359,0.855469,0.867188,0.833984,0.837573,0.842515,0.017072,5


In [ ]:
test_data  = pd.read_csv("resume.csv", sep=',')
test_data['job search'] = 0.0
test_data.head()

In [ ]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopword]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(test_data['summary'])
X_tfidf_feat = pd.DataFrame(X_tfidf.toarray())

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(test_data['summary'])
X_count_feat = pd.DataFrame(X_count.toarray())

In [ ]:
#gs_predict = gs.predict(test_data[['job search']])